In [ ]:
data_dir="/content/gdrive/MyDrive/daycon-fashion/dataset"

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cd /content/gdrive/MyDrive/daycon-bluehouse

In [ ]:
!pip install colab_ssh --upgrade
NGROK_TOKEN = '' # ngrok 토큰
PASSWORD = '1234' # 비밀번호 설정

from colab_ssh import launch_ssh
launch_ssh(NGROK_TOKEN, PASSWORD)

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install torch

     |████████████████████████████████| 46.9MB 60kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 29.2MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595711 sha256=57da7d3c1f423bb10f6fc98fbb6c28692e02742eb50be08766b5a2dd9517a559
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.2MB 29.4MB/s 
     |████████████████████████████████| 757kB 28.5MB/s 
     |████████████████████████████████| 901kB 43.4MB/s 
     |████████████████████████████████| 3.0MB 41.2MB/s 


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-m5oow0ob
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-m5oow0ob
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12773 sha256=5f6f57fac0d00d0cfbacab22c1201eceb53a9cdfc5bf903792656de2e0bb3d43
  Stored in directory: /tmp/pip-ephem-wheel-cache-763q6525/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from transformers import BertModel, BertTokenizer, BertConfig

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")
# device = 'cpu'
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
#hyper parameters
small_dataset=False
max_len=140

In [ ]:
!unzip /content/gdrive/MyDrive/daycon-bluehouse/BlueHouse.zip

Archive:  /content/gdrive/MyDrive/daycon-bluehouse/BlueHouse.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
# 학습용 데이터셋 불러오기
import pandas as pd
train_data = pd.read_csv('/content/gdrive/MyDrive/daycon-bluehouse/train.csv')
train_data.head()
if small_dataset:
  train_data=train_data.head()
train_data=list(zip(train_data['data'],train_data['category']))

In [ ]:
train_data.head()

,index,category,data
0,0,2,신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지...
1,1,0,학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등...
2,2,1,"빙상연맹, 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요.. 최근 동계올..."
3,3,1,"티비 12세,15세 관람가도 연령확인 의무화 하자.. 제기 에전에 티비를 보다가 잠..."
4,4,1,무더운 여름철엔 남성들도 시원한 자율복장을 해야. 무더운 여름철에는 남성들도 노넥타...


In [ ]:
label_map={0:'인권/성평등',1:'문화/예술/체육/언론',2:'육아/교육'}

In [ ]:
# Train / Test set 분리
from sklearn.model_selection import train_test_split

train, test = train_test_split(train_data, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))

train shape is: 32000
test shape is: 8000


In [ ]:
# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)


class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([str(i[sent_idx])]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# data_train = BERTDataset(train_data, 0, 1, tok, max_len, True, False)

using cached model


In [ ]:
# Setting parameters
max_len = 140 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 80
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 100
learning_rate = 5e-5
patience=3
data_train = BERTDataset(train, 0,1,tok,max_len, True, False)
data_test = BERTDataset(test, 0,1,tok, max_len, True, False)

# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2, drop_last=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2, drop_last=True)


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 3, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
  
# 모델 학습 시작
best_val_acc=0.0
best_val_loss=1000
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    

    #validation
    with torch.no_grad():
      print("Calculating validation results...")
      model.eval()# 평가 모드로 변경
      val_loss_items = []
      val_acc_items = []
      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          valid_length= valid_length
          label = label.long().to(device)
          out = model(token_ids, valid_length, segment_ids)
          loss_item = loss_fn(out, label)
          pred = torch.argmax(out, dim=-1)
        
          acc_item=(pred == label).sum().cpu().numpy()

          val_loss_items.append(loss_item)
          val_acc_items.append(acc_item)

      val_loss = np.sum(val_loss_items) / len(test_dataloader)
      val_acc = np.sum(val_acc_items) / len(test_dataloader)
      
      # Callback1: validation accuracy가 향상될수록 모델을 저장합니다.
      if val_loss < best_val_loss:
          best_val_loss = val_loss
      if val_acc > best_val_acc:
          print("New best model for val accuracy! saving the model..")
          best_val_acc = val_acc
          torch.save(model.state_dict(), f"/content/gdrive/MyDrive/daycon-bluehouse/model/{e:03}_accuracy_{best_val_acc:4.2%}_loss_{best_val_loss:4.4%}.pth")
          counter = 0
      else:
          counter += 1
      # Callback2: patience 횟수 동안 성능 향상이 없을 경우 학습을 종료시킵니다.
      if counter > patience:
          print("Early Stopping...")
          break
      
      
      print(
          f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
          f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2} || "
      )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


epoch 1 batch id 1 loss 1.2194421291351318 train acc 0.2375
epoch 1 batch id 101 loss 0.34598833322525024 train acc 0.7138613861386138
epoch 1 batch id 201 loss 0.3584938645362854 train acc 0.8033582089552236
epoch 1 batch id 301 loss 0.4025479257106781 train acc 0.8307724252491693

epoch 1 train acc 0.8493125000000002
Calculating validation results...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



New best model for val accuracy! saving the model..
[Val] acc : 6963.00%, loss: 0.36 || best acc : 6963.00%, best loss: 0.36 || 


epoch 2 batch id 1 loss 0.2936430871486664 train acc 0.8625
epoch 2 batch id 101 loss 0.2593693137168884 train acc 0.8887376237623761
epoch 2 batch id 201 loss 0.29718443751335144 train acc 0.9033582089552241
epoch 2 batch id 301 loss 0.3181746006011963 train acc 0.9053986710963462

epoch 2 train acc 0.9111250000000005
Calculating validation results...


...


New best model for val accuracy! saving the model..
[Val] acc : 7027.00%, loss: 0.38 || best acc : 7027.00%, best loss: 0.36 || 


epoch 4 batch id 1 loss 0.18091954290866852 train acc 0.9375
epoch 4 batch id 101 loss 0.1672091782093048 train acc 0.9376237623762381
epoch 4 batch id 201 loss 0.2075793296098709 train acc 0.9460820895522388
epoch 4 batch id 301 loss 0.19741646945476532 train acc 0.9493355481727571

epoch 4 train acc 0.9526249999999998
Calculating validation results...



New best model for val accuracy! saving the model..
[Val] acc : 7046.00%, loss: 0.42 || best acc : 7046.00%, best loss: 0.36 || 


epoch 5 batch id 1 loss 0.12522947788238525 train acc 0.9375
epoch 5 batch id 101 loss 0.1420130878686905 train acc 0.9574257425742565
epoch 5 batch id 201 loss 0.144870787858963 train acc 0.9631218905472642
epoch 5 batch id 301 loss 0.10101626813411713 train acc 0.9646179401993362

epoch 5 train acc 0.9669375000000011
Calculating validation results...



[Val] acc : 7030.00%, loss: 0.43 || best acc : 7046.00%, best loss: 0.36 || 


In [ ]:
torch.save(model.state_dict(), f"/content/gdrive/MyDrive/daycon-bluehouse/model/bert1.pth")

In [ ]:
# 학습용 데이터셋 불러오기
import pandas as pd
submission_data = pd.read_csv('/content/gdrive/MyDrive/daycon-bluehouse/test.csv')
submission_data.head()

submission_data=submission_data['data']


In [ ]:
# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

class testBERTDataset(Dataset):
    def __init__(self, dataset, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([str(i)]) for i in dataset]

    def __getitem__(self, i):
        return self.sentences[i]

    def __len__(self):
        return (len(self.sentences))

submission_data = testBERTDataset(submission_data, tok, max_len, True, False)

using cached model


In [ ]:
model_root="/content/gdrive/MyDrive/daycon-bluehouse/model/003_accuracy_7046.00%_loss_35.5215%.pth"
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
model.load_state_dict(torch.load(model_root))
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [ ]:
test_input = torch.utils.data.DataLoader(submission_data, batch_size=1, num_workers=2)
test_output=[]
for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(test_input)):
  with torch.no_grad():
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    out = model(token_ids, valid_length, segment_ids)
    pred = torch.argmax(out, dim=-1)
    test_output.extend(pred.cpu().numpy())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
import os
submission = pd.read_csv('/content/gdrive/MyDrive/daycon-bluehouse/sample_submission.csv')
submission['category'] = test_output
print(submission)
submission.to_csv(os.path.join('/content/gdrive/MyDrive/daycon-bluehouse', 'output2.csv'), index=False)

      index  category
0         0         0
1         1         2
2         2         1
3         3         0
4         4         2
...     ...       ...
4995   4995         0
4996   4996         1
4997   4997         1
4998   4998         0
4999   4999         2

[5000 rows x 2 columns]


In [ ]:
len(test_input)

5000

In [ ]:
len(submission)

5000